<a href="https://colab.research.google.com/github/yuchiahung/data-course-sample/blob/main/AC_hw3_MF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week3 -- Matrix Factorization (2022/01/03)

## 基礎建設

In [2]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [3]:
import numpy as np
import re
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

In [4]:
# from tqdm import tqdm

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

## 載入資料

In [6]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-01-08 16:09:32--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.3’

All_Beauty.csv.3    100%[===================>]  14.78M  21.2MB/s    in 0.7s    

2022-01-08 16:09:33 (21.2 MB/s) - ‘All_Beauty.csv.3’ saved [15499476/15499476]

--2022-01-08 16:09:33--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.2’

meta_All_Beauty.jso 100%[===================>]   9.85M  16.8MB/s    in 0.6s   

In [7]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [8]:
metadata.shape, ratings.shape

((32892, 19), (371345, 4))

In [9]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [10]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

In [11]:
# drop duplicates rows
metadata_clean = metadata.loc[metadata.astype(str).drop_duplicates().index]
# replace empty value as Nan
metadata_clean.replace('', np.nan, inplace = True)
# clean column `rank`
metadata_clean['rank'] = metadata_clean['rank'].str.replace('&amp;', '&')
metadata_clean['rank'].fillna('0', inplace = True)
metadata_clean['rank_category'] = [re.search('in (.*) \(', r).group(1) if r != '0' else None for r in metadata_clean['rank']]
metadata_clean['rank_num'] = [re.search('(.*) in .*', r).group(1) if r != '0' else None for r in metadata_clean['rank']]
metadata_clean['rank_num'] = metadata_clean['rank_num'].str.replace(',', '').astype(float)
# excluding category != 'Beauty & Personal Care'
metadata_clean = metadata_clean[metadata_clean.rank_category == 'Beauty & Personal Care']
# convert `price` to float
metadata_clean['price'].fillna('0', inplace = True)
metadata_clean['price'] = [re.search('\$(.*)', p).group(1) if re.search('\$(.*)', p) != None else None for p in metadata_clean['price']]
metadata_clean['price'] = metadata_clean['price'].str.replace(',', '').astype(float)
# drop useless columns
metadata_clean.drop(
    ['category', 'tech1', 'fit', 'tech2', 'date', 'similar_item', 'feature', 'main_cat', 'rank'], 
    axis = 1, 
    inplace = True
)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [12]:
### clean title & description
# join description list as string
metadata_clean['description'] = metadata_clean.description.apply(lambda x: ''.join(x))

for c in ('title', 'description'):
    # to lower case
    metadata_clean[c] = metadata_clean[c].apply(lambda x: x.lower())
    # replace &amp; as & 
    metadata_clean[c] = metadata_clean[c].str.replace('&amp;', '&')
    # replace &quot; as "
    metadata_clean[c] = metadata_clean[c].str.replace('&quot;', '"')

# clean html tag in description
metadata_clean['description_clean'] = metadata_clean.description.apply(lambda x: re.sub('<.*?>|\\n', '', x))

# remove strings in () [] & model info (i.e. number with oz, ml, cm)
metadata_clean['title_clean'] = metadata_clean.title.apply(lambda x: re.sub('\-\s.*?\s\-|\(.*?\)|\[.*?\]|\d{1,3}\.?\d{0,2}\s?(oz|z|ml|ea|g|cm|mm|fz|\-?ounce|count|ct|\"|fl\.? oz\.?|fluid ounce|x|\-?pac(k?)|\-?piece|pc(s?)|inch)|\d{1,2}\'d{0,2}|x\s?\d{1,3}|(set|pack) of \d{1,3}|colors may vary', '', x))
metadata_clean['title_clean'] = metadata_clean.title_clean.apply(lambda x: re.sub('[^\w\s]', '', x))

# title + description
metadata_clean['title_description'] = metadata_clean['title_clean'] + metadata_clean['description_clean']

# reset index
metadata_clean.reset_index(drop = True, inplace = True)

In [79]:
### clean brand name
metadata_clean['brand_clean'] = metadata_clean['brand'].apply(lambda x: str(x).lower())
metadata_clean['brand_clean'] = metadata_clean.brand_clean.apply(lambda x: re.sub('[^\w\s]', '', str(x)))

In [80]:
metadata_clean.brand.nunique(), metadata_clean.brand_clean.nunique()

(7734, 7462)

## 資料切分

In [15]:
# metadata_clean.to_csv('/content/metadata_clean.csv')
# ratings.to_csv('/content/ratings.csv')

In [16]:
# metadata_clean = pd.read_csv('/content/metadata_clean.csv', index_col = 9)
# ratings = pd.read_csv('/content/ratings.csv', index_col = 0)

In [17]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [18]:
ratings_trainings.shape

(370752, 5)

In [19]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

## Surprise: MF

In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 4.4 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619419 sha256=37e7428e1d547dfc2c1e4cc24a739c71c9ac1eb3057a650de55a3450f34202dd
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
from collections import defaultdict

def get_top_n(predictions, n=10):

    # 首先將預測資料對應到 user 
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # 將預測值進行排序，並且選取最高的 n 個值回傳
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

def recommend_items(model, user, items_user_not_rated, ratings_mean, k):
    
    # 取出測試資料
    user_recommend_data = [(user, item, ratings_mean) for item in items_user_not_rated]

    # 預測
    predictions = model.test(user_recommend_data)
    top_n = get_top_n(predictions, n=k)
    return [top[0] for top in top_n[user]]
    

In [ ]:
from scipy.sparse import csr_matrix
from surprise import Reader
from surprise import Dataset
from surprise import SVD, SVDpp
from surprise import accuracy
import numpy as np

def recommender(training_data, users=[], k=10, model_name = SVD):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    mf-based, surprise
    '''
    
    # 取出評分矩陣所需的訓練資料
    training_data = training_data[["asin","reviewerID","overall"]]
    item_set_trainings = set(training_data["asin"].to_list())

    # 轉換訓練資料為套件需要的格式
    reader = Reader(rating_scale = (min(training_data["overall"]), max(training_data["overall"])))
    train_data = Dataset.load_from_df(training_data, reader).build_full_trainset()

    # 訓練模型
    model = model_name()
    # model = SVD()
    # model = SVDpp()
    model.fit(train_data)

    # 推薦
    recommendations = {user: recommend_items(model, user, item_set_trainings - set(training_data.loc[training_data['reviewerID'] == user]['asin'].to_list()), training_data[["overall"]].mean(), k) for user in users}
    
    return recommendations


### SVD

In [ ]:
%%time
# 推薦成果：可看出冷啟動問題嚴重，針對未曾出現的使用者無法有鑑別的個別推薦，實務上可能會主動請使用者先評分再訓練進行推薦
ratings_by_user = recommender(ratings_trainings, users)

CPU times: user 4min, sys: 738 ms, total: 4min 1s
Wall time: 4min 1s


In [ ]:
print('Surprise - SVD score: ')
evaluate(ratings_testings_by_user, ratings_by_user)

0.0

### SVDpp

In [ ]:
%%time
ratings_by_user_SVDpp = recommender(ratings_trainings, users, model_name = SVDpp)

CPU times: user 1h 10min 32s, sys: 9.33 s, total: 1h 10min 41s
Wall time: 1h 10min 41s


In [ ]:
print('Surprise - SVDpp score: ')
evaluate(ratings_testings_by_user, ratings_by_user_SVDpp)

Surprise - SVDpp score: 


0.0

### NMF

In [ ]:
from surprise import NMF

In [ ]:
%%time
ratings_by_user_NMF = recommender(ratings_trainings, users, model_name = NMF)

CPU times: user 4min 35s, sys: 628 ms, total: 4min 35s
Wall time: 4min 35s


In [ ]:
print('Surprise - NMF score: ')
evaluate(ratings_testings_by_user, ratings_by_user_NMF)

Surprise - NMF score: 


0.0

## LightFM: MF

In [20]:
!pip install lightfm

In [21]:
from lightfm import LightFM
import numpy as np

def recommend_items(model, data, user_id, item_list, k):
    
    # 取得item數量
    n_users, n_items = data.shape

    # 預測user對所有item的評分並進行推薦
    scores = model.predict(user_id, np.arange(n_items))
    top_items = item_list[np.argsort(-scores)]
    return top_items[:k].tolist()

In [91]:
from lightfm.data import Dataset

def recommender_lightfm(training_data, users=[], k=10, item_metadata=None):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    mf-based, lightFM
    '''
    
    # 取出評分矩陣所需的訓練資料並以lightfm的Dataset形式存取
    training_data = training_data[["reviewerID","asin","overall"]]
    data = Dataset()
    data.fit(np.append(training_data.reviewerID.unique(), users, 0), training_data.asin.unique())
    # create a (num_users, num_items) COO matrix with interactions, and a (num_users, num_items) matrix with the corresponding interaction weights.
    interactions_matrix, weights_matrix = data.build_interactions([tuple(i) for i in training_data.values])
    if item_metadata is not None:
        # Build a item features matrix out of an iterable of the form (item id, [list of feature names]) or (item id, {feature name: feature weight}).
        for i in range(2, 6):
            data.fit_partial(items=(x[1] for x in item_metadata.itertuples()),
                            item_features=(x[i] for x in item_metadata.itertuples()))
        item_features = data.build_item_features((
            (x[1], list(x[2:]))
            for x in item_metadata.itertuples()
            ))

    # 取得user與item的mapping資料
    user_id_map, user_feature_map, item_id_map, item_feature_map = data.mapping()
    item_list = np.array(list(item_id_map.keys())).astype(object)
    
    # 訓練模型
    model = LightFM(loss='warp', item_alpha=1e-6, no_components=30)

    if item_metadata is not None: # 訓練混合模型（Hybrid Model）加入 user / item feature
        model = model.fit(interactions_matrix, sample_weight = weights_matrix, 
                          item_features=item_features, epochs=3, num_threads=2)
    
    else:
        model = model.fit(interactions_matrix, sample_weight = weights_matrix, 
                          epochs=3, num_threads=2)

    # 推薦
    recommendations = {
        user: recommend_items(model, interactions_matrix, user_id_map[user], item_list, k)
        for user in users}

    return recommendations


In [20]:
%%time
# 推薦成果：可看出冷啟動問題嚴重，針對未曾出現的使用者無法有鑑別的個別推薦，實務上可能會主動請使用者先評分再訓練進行推薦
#         除此之外也能嘗試使用混合模型（Hybrid Model）加入user / item feature進行訓練
ratings_by_user = recommender_lightfm(ratings_trainings, users)

CPU times: user 12.4 s, sys: 84.7 ms, total: 12.5 s
Wall time: 10.3 s


In [ ]:
# ratings_by_user

In [21]:
print('Sample LightFM MF score:')
evaluate(ratings_testings_by_user, ratings_by_user)

Sample LightFM MF score:


0.08305084745762711

### LightFM + item_features

In [69]:
metadata_clean['price_cat'] = pd.qcut(metadata_clean.price, 5, labels=False).fillna(2)
metadata_clean['rank_num_cat'] = pd.qcut(metadata_clean.rank_num, 5, labels=False).fillna(2)
item_metadata_clean = metadata_clean[['asin', 'price_cat', 'brand_clean', 'rank_num_cat', 'title_clean']]

In [75]:
%%time
ratings_by_user = recommender_lightfm(ratings_trainings, users, item_metadata=item_metadata_clean)

CPU times: user 19.2 s, sys: 88.2 ms, total: 19.3 s
Wall time: 14.2 s


In [76]:
print('LightFM MF + item_features score:')
evaluate(ratings_testings_by_user, ratings_by_user)

LightFM MF + item_features score:


0.08305084745762711